In [1]:
import os
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [25]:
#------- User Inputs -----------------------
#save path and output filename
savepath='raw/'
filename = 'prosportstransactions_scrape_inactivelist_2019_2023.csv'


#URL to scrape from
url = "https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search"
#Note that only the first subset of results is shown on this webpage. The remainder of the data is broken out into a number of web pages that are linked at the bottom. 

In [26]:
#-------------Scrape web page----------------------------

#Get URL HTML
response = requests.get(url)
print(response) # Response [200] means it went through

#Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

<Response [200]>


In [27]:
#-------------Scrape data from the first page----------------
#Read in html as pandas data frame
df_first_page = pd.read_html(response.text)
    
#Select table of interest (the first table)
df_first_page = df_first_page[0]

#Drop first row (column names)
df_first_page.drop([0], inplace = True)
   
#Remove bullet in front of names
df_first_page[2]=df_first_page[2].str[2:] # "Acquired" column
df_first_page[3]=df_first_page[3].str[2:] # "Relinquished" column
    
#Modify column titles
df_first_page.columns = ['Date','Team','Acquired','Relinquished','Notes']

#dataframe to hold data
appended_data = df_first_page

In [28]:
df_first_page

,Date,Team,Acquired,Relinquished,Notes
1,2019-07-01,76ers,NaN,Amir Johnson,placed on IL
2,2019-10-01,Wizards,NaN,John Wall (Hildred),placed on IL recovering from torn left Achille...
3,2019-10-21,Raptors,NaN,Patrick McCaw,placed on IL with sore left knee
4,2019-10-21,Raptors,NaN,Rondae Hollis-Jefferson,placed on IL with left groin injury
5,2019-10-22,Clippers,NaN,Paul George,placed on IL with right shoulder injury
6,2019-10-22,Clippers,NaN,Rodney McGruder,placed on IL with sprained right ankle
7,2019-10-22,Lakers,NaN,DeMarcus Cousins,placed on IL recovering from surery on left kn...
8,2019-10-22,Lakers,NaN,Kyle Kuzma,placed on IL with stress reaction in left ankl...
9,2019-10-22,Lakers,NaN,Rajon Rondo,placed on IL with right calf injury
10,2019-10-22,Pelicans,NaN,Zion Williamson,placed on IL recovering from surgery on right ...


In [29]:
#------------Scrape data from other pages linked at the bottom of the first page------------
# Loop over links (skipping the first 4 (not data) and last 4 ("Next" and other webpage links))
for i in range(4,len(soup.findAll('a'))-4): #'a' tags are for links
   
    #find all links on webpage and select the i-th link
    one_a_tag = soup.findAll('a')[i]
    link = one_a_tag['href']
    
    #Add in the rest of the url
    download_url = 'https://www.prosportstransactions.com/basketball/Search/'+ link
    print(download_url)
    this_response = requests.get(download_url)
    
    #Read html as pandas data frame
    # dfs = pd.read_html(download_url)
    dfs = pd.read_html(this_response.text)
    
    #Select table of interest (the first table)
    df = dfs[0]
    
    #Drop first row (column names)
    df.drop([0], inplace = True)
   
    #Remove bullet in front of names
    df[2]=df[2].str[2:] # "Acquired" column
    df[3]=df[3].str[2:] # "Relinquished" column
    
    #Modify column titles
    df.columns = ['Date','Team','Acquired','Relinquished','Notes']
    
    #Append data frame 
    appended_data=pd.concat([appended_data,df], ignore_index=True)
    
    #Add a pause to keep web server happy
    time.sleep(5)

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=25
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=50
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=75
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=100
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=125
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=150
https://www.prospor

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1275
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1300
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1325
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1350
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1375
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=1400
https://ww

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2525
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2550
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2575
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2600
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2625
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=2650
https://ww

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3775
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3800
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3825
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3850
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3875
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=3900
https://ww

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5025
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5050
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5075
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5100
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5125
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=5150
https://ww

https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6275
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6300
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6325
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6350
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6375
https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2019-07-01&EndDate=2023-03-01&ILChkBx=yes&Submit=Search&start=6400
https://ww

In [30]:
appended_data.shape[0]

7311

In [31]:
#-------------------Save file----------------------------------------------
    
print('Saving files......')

filename= os.path.join(savepath,filename)
appended_data.to_csv(filename)

print('Finished')

Saving files......
Finished


In [32]:
il_data = pd.read_csv('raw/prosportstransactions_scrape_inactivelist_2010_2023.csv')

In [34]:
il_data

,Date,Team,Acquired,Relinquished,Notes
0,10/26/2010,Blazers,NaN,Elliot Williams,placed on IL
1,10/26/2010,Blazers,NaN,Greg Oden,placed on IL with left knee injury (out for se...
2,10/26/2010,Blazers,NaN,Joel Przybilla,placed on IL placed on IL recovering from surg...
3,10/26/2010,Celtics,NaN,Avery Bradley,placed on IL recovering from surgery on left a...
4,10/26/2010,Celtics,NaN,Kendrick Perkins,placed on IL recovering from surgery on right ...
...,...,...,...,...,...
21439,2/25/2023,Pelicans,Dyson Daniels,NaN,activated from IL
21440,2/25/2023,Pistons,Marvin Bagley III,NaN,activated from IL
21441,2/26/2023,Hawks,John Collins (Martin),NaN,activated from IL
21442,2/26/2023,Nuggets,Aaron Gordon,NaN,activated from IL
